In [3]:
import torch
import gym

In [4]:
# Creating Envrionment
env = gym.make("FrozenLake-v0", is_slippery=False)

In [5]:
import numpy as np
# one hot encoder
# input: observation spaces from gym
# ouput: one-hot encoded vector (torch tensor)
def one_hot(i):
    state = np.zeros(shape=(1,env.observation_space.n),dtype=int)
    state[0][i] = 1 
    return torch.Tensor(state) # pytorch compatible variables

one_hot(3).shape

torch.Size([1, 16])

In [11]:
model = nn.Sequential(
    nn.Linear(16, 12),
    nn.ReLU(),
    nn.Linear(12,4) #
)

model(one_hot(3)).squeeze().argmax().item()

3

In [18]:
## Initialize DQN with pytorch
import torch
import torch.nn as nn
from collections import deque 

## deep learning architecture for DQN
model = nn.Sequential(
    nn.Linear(16, 12),
    nn.ReLU(),
    nn.Linear(12,4) #
)

## define optimizer
optimizer = torch.optim.Adam(model.parameters())

## hyper parameters
discount_factor = 0.95
epsilon = 0.6
epsilon_decay_factor = 0.999
num_episodes = 500

def DQN(model, optimizer, discount_factor, epsilon, epsilon_decay_factor, num_episodes):
    ## Initialize the Q-network
    ## Iterate from 1...M episodes
    avg_reward = deque(maxlen=100)


    for episode in range(num_episodes+1):
        ## Reset the game, initialize the states
        state = env.reset()
        
        ## repeat process until end of the epsidoes
        done = False
        epsilon = epsilon * epsilon_decay_factor

        reward_list = []
        while not done:
            ## with prob. epsilon take random action
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else: #otherwise, select best action w.r.t Q-network
                action = model(one_hot(state)).squeeze().argmax().item()
            # execute action and observe new status, and reward
            next_state, reward, done, _ = env.step(action)
            # calcualte target value(y)
            if done:
                target = reward
            else:
                target = reward + discount_factor*model(one_hot(next_state)).squeeze().max()
            # calcualte pred
            pred = model(one_hot(state)).squeeze().max()
            # loss function
            loss = (target-pred).pow(2)

            # perform a gradient descent
            optimizer.zero_grad() # initialze the optimizer with zero
            loss.backward() # calculate the gradient of loss function
            optimizer.step() # we update the weight of the model using gradient

            # update the state
            state = next_state

            # put reward to list
            reward_list.append(reward)

            if episode == num_episodes:
                env.render()

        avg_reward.append(sum(reward_list))
        if episode%100 == 0: # for every 100 steps, print some info
            print(episode, np.mean(avg_reward), epsilon)


DQN(model, optimizer, 0.95, 0.8, 0.999, 2000)
        


            





0 0.0 0.7992
100 0.01 0.7231098839732764
200 0.01 0.6542641445193258
300 0.03 0.5919730600991551
400 0.12 0.5356125760805872
500 0.24 0.48461805273305963
600 0.22 0.4384786084624098
700 0.48 0.3967320016141349
800 0.44 0.3589599995691724
900 0.44 0.32478418873812753
1000 0.63 0.2938621834769546
1100 0.62 0.26588419594363666
1200 0.64 0.24056993253144524
1300 0.6 0.2176657857861265
1400 0.68 0.19694229367462207
1500 0.64 0.1781918407513601
1600 0.62 0.16122657819156724
1700 0.77 0.14587654185374446
1800 0.74 0.13198794951737225
1900 0.79 0.11942166023695906
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
2000 0.82 0.108051780377682
